In [148]:
import numpy as np
import random as rd
import time
import matplotlib.pyplot as plt
from scipy.constants import k
plt.rcParams['figure.dpi'] = 300

# USER INPUT PARAMETERS ######################################################################################

# VdW parameters
epsilon = 0.0103
sigma = 3.4 
alpha = 1
r_cut = 8


# box and system initialisation parameters
rho = 0.04
T = 0.1 # temp of system initially (K)
m = 1
lx = 10 # box dimension (units of sigma)
ly = lx
N = round(lx*ly*rho)
lattice_spacing = sigma*2**(1/6) # effectively the particle radius - lattice spacing is the closest two particles can spawn


# Duration of Simulation
Nts = 10 # number of time steps
dt = 0.000001 # length of time step


# Energy/Temp File saving parameters
timestring = time.strftime("%Y%m%d-%H%M%S")
fname = 'logfile_{}.txt'.format(timestring)
flog = 20 

# Trajectory File saving parameters
fname_traj_unwrapped = 'traj_uw{}.gro'.format(timestring)
fname_traj_wrapped = 'traj_w{}.gro'.format(timestring)




In [149]:
# INITIALISE THE SYSTEM POSITIONS AND VELOCITIES ############################################################
def Initialise_Positions(N, lx, ly, lattice_spacing):
    x = np.zeros(N)
    y = np.zeros(N)

    for i in range(N):
        while True:
            # generate random positions
            new_position_x = np.random.randint(1, lx-1)
            new_position_y = np.random.randint(1, ly-1)

            # check for overlaps with existing particles
            overlap = False
            for j in range(i):
                if np.linalg.norm([new_position_x - x[j], new_position_y - y[j]]) < lattice_spacing:
                    overlap = True
                    break

            # if no overlap is found, assign the new position and break the loop
            if not overlap:
                x[i] = new_position_x
                y[i] = new_position_y
                break

    # return the grid scaled with grid spacing sigma
    # note: if sigma is not set to an appropriate value, the LJ force calculated will be either too big or too small
    return x * lattice_spacing, y * lattice_spacing


def Initialise_Velocities(N, T):
    # initialise x velocities acc to the Boltzmann distribution
    vx = np.zeros(N) #array
    vy = np.zeros(N) 
    for i in range(N):
        vx[i] = rd.uniform(-1.0,1.0) #float (element of vx)
        vy[i] = rd.uniform(-1.0,1.0)

    # calculate the average velocities
    vx_avg = np.average(vx)
    vy_avg = np.average(vy) 
    v_avg2 = np.sum((vx*vx)+(vy*vy))

    # calculate scale factor according to the formula
    f_s = np.sqrt( (2*T)/(v_avg2) ) 
    

    # scale the velocities to remove initial drift
    vx = (vx - vx_avg)*f_s
    vy = (vy - vy_avg)*f_s

    return vx, vy # [*] and vy*fs_y

In [150]:
#   CALCULATE FORCES BETWEEN ALL INTERACTING PARTICLES ####################################################
def LJ_Force(r): # returns absolute force based upon absolute distance between the particles
    LJF = 0
    if r > r_cut:
        return 0 
    else:
       LJF = 24 * epsilon * (2 * (sigma**12 / r**13) - alpha*(sigma**6 / r**7))
       return LJF
    

def Calc_Accel(x_pos, y_pos): # returns the instantaneous accelerations F/m at each time step
    a_x = np.zeros((N,N)) # Initialise matrix to hold x accelerations
    a_y = np.zeros((N,N)) # matrix for y accelerations

    # calculate only the 'top half' of the matrix of forces between particles i and j
    for i in range(0, N-1):
        for j in range(i+1, N):
            r_x = x_pos[i] - x_pos[j]
            r_x = r_x % lx # wrap the distance [?]
            r_y = y_pos[i] - y_pos[j]
            r_y = r_y % ly
            r = np.sqrt(r_x*r_x + r_y*r_y) 


            F = LJ_Force(abs(r))
            F_x = F * abs(( r_x / r ))
            F_y = F * abs(( r_y / r ))


            a_x[i,j] = F_x/m # forces on particles are equal and opposite
            a_x[j,i] = -F_x/m
            a_y[i,j] = F_y/m 
            a_y[j,i] = -F_y/m

    acc_x = np.sum(a_x,axis=0)
    acc_y = np.sum(a_y,axis=0)

    return acc_x, acc_y

In [151]:
# VERLET INTEGRATION OF POSITION AND VELOCITIY ############################################################
def Verlet_Pos(x_prev, x_curr, ax): 
    x_next = 2*x_curr - x_prev + ax*(dt*dt)
    return x_next
    
def Verlet_Vel(x_prev, x_next):
    vx = (1/(2*dt)) * (x_next.copy() - x_prev.copy())
    return vx

In [152]:
# CALCULATE INSTANTANEOUS TEMPERATURE, KINETIC ENERGY, POTENTIAL ENERGY AND TOTAL ENERGY ##################

def Calc_Temp(vx, vy):
    v_squared = vx*vx + vy*vy
    T = ((m)/4)*np.sum(v_squared)
    # I have omitted the factor of k from the formula since we have set m=1 and the temperature calculated will be scaled down otherwise
    # The temperature calculated using this method was tested using the initial configuration and, while it wasn't exactly equal to T because we are using a distribution
    # of velocities and a small number of particles, it was at least of the same order
    return T

def Calc_KE(vx, vy):
    v_squared = vx*vx + vy*vy
    KE = 0.5*m*np.sum(v_squared)
    return KE

def LJ_Potential(r):
    if r > r_cut:
        return 0 
    else:
       LJP = 4 * epsilon * ((sigma**12 / r**12) - alpha*(sigma**6 / r**6))
       
       return LJP
    
def Calc_PE(x_pos, y_pos):
    PE = 0
    for i in range(0, N-1):
        for j in range(i+1, N):
            r_x = x_pos[i] - x_pos[j]
            r_x = r_x % lx # wrap the distance [?]
            r_y = y_pos[i] - y_pos[j]
            r_y = r_y % ly
            r = np.sqrt(r_x*r_x + r_y*r_y) 

            PE = PE + LJ_Potential(r) # check lol

    return PE

In [153]:
##################################################################################################################
# INITIALISE SYSTEM ##############################################################################################
##################################################################################################################


x_init, y_init = Initialise_Positions(N,lx,ly,lattice_spacing)
vx_init, vy_init = Initialise_Velocities(N, T)
ax_init, ay_init = Calc_Accel(x_init, y_init)

Temps = np.zeros(Nts)
# in the Euler approximation, the velocity is constant for the first two timesteps, therefore so is the temperature
Temps[0] = T
Temps[1] = T

PEs = np.zeros(Nts)
PEs[0] = Calc_PE(x_init, y_init)

KEs = np.zeros(Nts)
KEs[0] = Calc_KE(x_init, y_init)


TE = KEs[0] + PEs[0]
data = [0, Temps[0], KEs[0], PEs[0], TE]

with open(fname, "w") as file:
            file.write("Timestep, Temperature, Kinetic Energy, Potential Energy, Total Energy\n")
            file.write(",".join(map(str, data)) + "\n")


# initialise coordinate- and velocity-holding matrices such that the row represents the timestep and the column is the particle in the system
# We will perform the first two timesteps after initialisation under constant velocity since we need the previous and next values to calculate current velocity
x_unwrapped = np.zeros((Nts+1, N)) # this contains an extra timestep to fit the final 'next position' when calculating the final velocity, but we'll only plot up to Nts
x_wrapped = np.zeros((Nts+1, N))
x_unwrapped[0, :] = x_init
x_wrapped[0, :] = x_init
vx = np.zeros((Nts, N))
vx[0, :] = vx_init
vx[1, :] = vx_init


y_unwrapped = np.zeros((Nts+1, N))
y_wrapped = np.zeros((Nts+1, N))
y_unwrapped[0, :] = y_init
y_wrapped[0, :] = y_init
vy = np.zeros((Nts, N))
vy[0, :] = vy_init
vy[1, :] = vy_init


##################################################################################################################
# START SIMULATION ###############################################################################################
##################################################################################################################

#print('0')

Temps[0] = Calc_Temp(vx[0,:],vy[0,:])
#print(Temps[0])

KEs[0] = Calc_KE(vx[0,:],vy[0,:])
#print(KEs[0])

PEs[0] = Calc_PE(x_unwrapped[0,:], y_unwrapped[0,:])
#print(PEs[0])
        

# perform the first two iterations using an Euler approximation (constant velocity) just to get things started
x_unwrapped[1, :] = x_unwrapped[0, :] + vx[0, :]*dt + 0.5*ax_init*dt*dt
y_unwrapped[1, :] = y_unwrapped[0, :] + vy[0, :]*dt + 0.5*ay_init*dt*dt


# forces are updated but not velocity, which stays constant for first two timesteps
ax, ay = Calc_Accel(x_unwrapped[1,:], y_unwrapped[1,:])

x_unwrapped[2, :] = x_unwrapped[1, :] + vx[1, :]*dt + 0.5*ax*dt*dt
y_unwrapped[2, :] = y_unwrapped[1, :] + vy[1, :]*dt + 0.5*ay*dt*dt



# Now we may use Verlet integration as normal!

for ts in range(2,Nts):
    # calculate the force on the current configuration
    ax, ay = Calc_Accel(x_unwrapped[ts,:], y_unwrapped[ts,:])
    
    # due to this force, calculate the new positions of the particles for the next timestep
    x_unwrapped[ts+1,:] = Verlet_Pos(x_unwrapped[ts-1,:], x_unwrapped[ts,:], ax)
    y_unwrapped[ts+1,:] = Verlet_Pos(y_unwrapped[ts-1,:], y_unwrapped[ts,:], ay)

    # using the next and previous positions, calculate the velocity for the current timestep
    vx[ts,:] = Verlet_Vel(x_unwrapped[ts-1,:], x_unwrapped[ts+1,:])
    vy[ts,:] = Verlet_Vel(y_unwrapped[ts-1,:], y_unwrapped[ts+1,:])

    if ts % flog == 0:
        
        #print(ts)

        Temps[ts] = Calc_Temp(vx[ts,:],vy[ts,:])
        #print(Temps[ts])

        KEs[ts] = Calc_KE(vx[ts,:],vy[ts,:])
        #print(KEs[ts])
        
        PEs[ts] = Calc_PE(x_unwrapped[ts,:], y_unwrapped[ts,:])
        #print(PEs[ts])


        # write to file current timestep, instantaneous temp, KE, PE and TE
        TE = KEs[ts] + PEs[ts]
        data = [ts, Temps[ts], KEs[ts], PEs[ts], TE]

        with open(fname, "a") as file:
            file.write(",".join(map(str, data)) + "\n")


x_wrapped = x_unwrapped.copy() % lx
y_wrapped = y_unwrapped.copy() % ly 
# at the end of the simulation, write all the coordinates to the trajectory .gro file
            
# with open(fname_traj_unwrapped, 'w') as file:
#     file.write("MD Sim; {}/n".format(timestring))
#     file.write(str(N) + "/n")

p=0
t=0



for t in range(Nts):
    print('timestep: ' + str(t))
    for p, (x, y) in enumerate(zip(x_unwrapped[t,:], y_unwrapped[t,:])):
        print('particle: ' + str(p))
        print((x,y))




with open(fname_traj_wrapped, 'w') as file:
    file.write("MD Sim; {}/n".format(timestring))
    file.write(str(N) + "\n")
    for t in range(Nts):
        for p, (x, y) in enumerate(zip(x_unwrapped[t,:], y_unwrapped[t,:])):
            file.write(f"{i+1:5d}{'LJ':5s}{'C':5s}{i+1:5d}{x:8.3f}{y:8.3f}{0.000:8.3f}\n")

             
        #file.write("{:8.4f}{:8.4f}{:8.4f}\n".format(lx, ly, 0.0))
         
    
print('x unwrapped')
print(x_unwrapped)
print('y unwrapped')
print(y_unwrapped)            

    # This loop will fill up the positions matrices x and y for Nts+1 timesteps
    # the velocity matrices will be filled up to Nts (because we need ts+1 and ts-1 for the current velocity)






timestep: 0
particle: 0
(26.71459674976308, 26.71459674976308)
particle: 1
(7.632741928503736, 7.632741928503736)
particle: 2
(22.898225785511208, 3.816370964251868)
particle: 3
(11.449112892755604, 30.530967714014945)
timestep: 1
particle: 0
(26.714596809805958, 26.714596550329656)
particle: 1
(7.6327416477096595, 7.632741912907625)
particle: 2
(22.898225874973257, 3.8163709680866287)
particle: 3
(11.449113024044753, 30.530967925209715)
timestep: 2
particle: 0
(26.714596869848837, 26.714596350896233)
particle: 1
(7.632741366915583, 7.632741897311514)
particle: 2
(22.898225964435305, 3.816370971921389)
particle: 3
(11.449113155333901, 30.530968136404486)
timestep: 3
particle: 0
(26.71459692989172, 26.714596151462814)
particle: 1
(7.632741086121507, 7.632741881715404)
particle: 2
(22.89822605389734, 3.816370975756126)
particle: 3
(11.449113286623058, 30.530968347599277)
timestep: 4
particle: 0
(26.714596989934606, 26.7145959520294)
particle: 1
(7.6327408053274315, 7.632741866119295)
par

In [154]:
p=0
t=0
for t in range(Nts):
    print('timestep: ' + str(t))
    for p, (x, y) in enumerate(zip(x_unwrapped[t,:], y_unwrapped[t,:])):
        print('particle: ' + str(p))
        print((x,y))

timestep: 0
particle: 0
(26.71459674976308, 26.71459674976308)
particle: 1
(7.632741928503736, 7.632741928503736)
particle: 2
(22.898225785511208, 3.816370964251868)
particle: 3
(11.449112892755604, 30.530967714014945)
timestep: 1
particle: 0
(26.714596809805958, 26.714596550329656)
particle: 1
(7.6327416477096595, 7.632741912907625)
particle: 2
(22.898225874973257, 3.8163709680866287)
particle: 3
(11.449113024044753, 30.530967925209715)
timestep: 2
particle: 0
(26.714596869848837, 26.714596350896233)
particle: 1
(7.632741366915583, 7.632741897311514)
particle: 2
(22.898225964435305, 3.816370971921389)
particle: 3
(11.449113155333901, 30.530968136404486)
timestep: 3
particle: 0
(26.71459692989172, 26.714596151462814)
particle: 1
(7.632741086121507, 7.632741881715404)
particle: 2
(22.89822605389734, 3.816370975756126)
particle: 3
(11.449113286623058, 30.530968347599277)
timestep: 4
particle: 0
(26.714596989934606, 26.7145959520294)
particle: 1
(7.6327408053274315, 7.632741866119295)
par

In [155]:

for t in range(Nts):
    print(x_unwrapped[t,p])

11.449112892755604
11.449113024044753
11.449113155333901
11.449113286623058
11.449113417912224
11.4491135492014
11.449113680490584
11.449113811779776
11.449113943068978
11.449114074358189


x_prev = np.zeros(N)
x_curr = Initialise_Positions(N,lx,ly)
x_next = np.zeros(N)

vx = Initialise_Velocities()
ax = Calc_Accel(x_curr.copy())
ax_next = np.zeros(N)

xs = np.zeros((Nts, N))
xs[0, :] = x_curr

axs = np.zeros((Nts, N))
axs[0, :] = ax

print('initial:')
print(x_prev)
print(x_curr)
print(x_next)
print(vx)
print(ax)

# perform the first iteration using an Euler approximation (constant velocity) just to get things started
# do once: x_prev = 0, x_curr = init, x_next = 0
x_next = x_curr.copy() + vx.copy()*dt + 0.5*ax.copy()*(dt**2)
# x_next = next, x_curr = init, x_prev = 0

x_prev = x_curr.copy()
x_curr = x_next.copy()
xs[1, :] = x_curr.copy()

ax = Calc_Accel(x_curr.copy())
axs[1, :] = ax

print('ts=0: ')
print(x_prev)
print(x_curr)
print(x_next)
print(vx)
print(ax)

# do again: x_prev = init, x_curr = next, x_next = next
x_next = x_curr.copy() + vx.copy()*dt + 0.5*ax.copy()*(dt**2) 
# x_next = next2, x_curr = next, x_prev = init

# now we have enough information to update the velocity!
vx = (1/(2*dt)) * (x_next.copy() - x_prev.copy())

x_prev = x_curr.copy()
x_curr = x_next.copy()
xs[2, :] = x_curr.copy()

ax = Calc_Accel(x_curr.copy())
axs[2, :] = ax

print('ts=1: ')
print(x_prev)
print(x_curr)
print(x_next)
print(vx)
print(ax)

for ts in range(1, Nts-1):
    xs[ts+1, :] = Verlet_Pos(xs[ts, :], xs[ts-1, :], ax)
    xs[ts+1, :] = xs[ts+1, :] % lx
    
    vx = Verlet_Vel(xs[ts-1, :],xs[ts+1, :])

    ax = Calc_Accel(xs[ts+1, :])
    axs[ts+1, :] = ax

    if ts % flog == 0:
        print(ts)
        print(xs[ts,:])
        print(vx)
        print(ax)
    

for i in range(N):
    plt.plot(xs[:, i], label='atom {}'.format(i))

plt.legend()
plt.ylim(0, lx)
plt.show()

for i in range(N):
    plt.plot(axs[:, i], label='atom {}'.format(i))

plt.legend()
plt.show()

# 1D case
---------------
### 19/12 Morning
- Wrote Initialise_Positions, Initialise_Velocities, LJ_Force, Calc_Accel <br> 
- The distances between positions are of order $10^0$ while the velocities are of order $10^{-12}$ - they are too slow moving and far apart to interact with each other
- An appropriate value for $\sigma$ needs to be chosen or the velocities scaled up or down

### 19/12 Evening
- Wrote Verlet_Pos
- Wrote first two Euler passes and plotted <br>
- The particles stay mainly on the same path, which is to be expected from such small interactions and small timescales. The positions do change very incrementally, just not enough to be visibly plotted. In the case an atom has two equidistant neighbours, there is no net force (which we expect). In the case two atoms are close and one is far away, the close atoms experience a force and the far away one does not (which we expect). I am happy to proceed further based on my code working (seemingly) properly so far. <br>

- Wrote Verlet_Vel
- Iterated over Nts timesteps, the code runs and plots but the particles do not behave as expected and aren't interacting strongly. Artificially placing two particles very close together doesn't make them explode apart, they just float along next to each other. <br>
- The smaller I make the box, the more likely I get errors related to r = 0 between two particles. It looks like they zoom away from each other symmetrically, hit the edge of the box, the coordinates are wrapped and then suddenly they are in the same place. They shouldn't be able to get to the edge of the box at the same time though, they should 'feel' each other and bounce away if they reach one edge and a particle is at the opposite edge.

## To Do:
1. fix particle motion issue - go through integration
2. try not having 3 arrays at all times, this is confusing with updating them - just use one giant matrix for the time being and literally call index [ts-1], etc...


### 21/12 Morning
- Changed intergration to use [ts,:] in matrix rather than keep track of 3 arrays
- particles don't interact
- if change $\sigma$ to ~ 1e-12, particles can spawn very close to each other and in this case bounce away immediately, otherwise, don't interact
- If use F instead of F_x, code runs for Nts>1000
- Changed F to F_x -> I put abs(r) into the LJ potential then The code runs and plots
- I changed all distances r to abs since the LJ force should account for - or + as needed. The particles attract each other when far away, but do not repel when they get close. They continually accelerate towards each other and, as the cell coordinates are wrapped, just shoot through each other again and again without repelling.
- Try plotting the force on one particle against time and comparing it to how it moves in that same time?



### 04/01
- Need to create arrays to hold wrapped and unwrapped coordinates
- Velocities should be calculated using UNWRAPPED COORDINATES
- Forces should be calculated using WRAPPED COORDINATES